# Phase 3 – Analysis and Recommendations
**Goal:** Link UHI areas with infrastructure deficit by identifying LST hotspots that fall outside buffered cooling infrastructure zones.

In [ ]:
import geopandas as gpd
import rasterio
from rasterio.features import shapes
from shapely.geometry import shape
import numpy as np
import folium
import json

# Load LST raster
with rasterio.open("/content/drive/MyDrive/urban-heat-dashboard/assets/pune_LST_C_mar_may2024.tif") as src:
    arr = src.read(1)
    transform = src.transform

# Define threshold for hotspot (e.g. > 35°C)
mask = arr > 35
results = (
    {"properties": {"value": v}, "geometry": s}
    for s, v in shapes(arr, mask=mask, transform=transform)
)

# Convert to GeoDataFrame
hotspot_gdf = gpd.GeoDataFrame.from_features(results, crs=src.crs)
hotspot_gdf = hotspot_gdf.to_crs(epsg=4326)
hotspot_gdf.to_file("/content/drive/MyDrive/urban-heat-dashboard/assets/pune_hotspots_35C.geojson", driver="GeoJSON")

In [ ]:
# Load the 200m Cooling Buffer Layer
cooling_buffer_gdf = gpd.read_file("/content/drive/MyDrive/urban-heat-dashboard/assets/pune_cooling_buffer_200m.geojson")
cooling_buffer_gdf = cooling_buffer_gdf.to_crs(epsg=4326)

In [ ]:
# Find Hotspots Outside Cooling Buffer
uhi_without_infra = gpd.overlay(hotspot_gdf, cooling_buffer_gdf, how="difference")
uhi_without_infra.to_file("/content/drive/MyDrive/urban-heat-dashboard/assets/pune_hotspots_outside_buffer.geojson", driver="GeoJSON")

In [ ]:
# Basic stats
print("Total hotspot polygons:", len(hotspot_gdf))
print("Hotspot polygons outside cooling buffer:", len(uhi_without_infra))

In [ ]:
# Visualization using folium
m = folium.Map(location=[18.52, 73.85], zoom_start=12)

# Add hotspot polygons
folium.GeoJson(hotspot_gdf, name="Hotspots > 35C", style_function=lambda x: {
    'fillColor': 'red', 'color': 'red', 'weight': 1, 'fillOpacity': 0.4
}).add_to(m)

# Add cooling buffer
folium.GeoJson(cooling_buffer_gdf, name="Cooling Buffer 200m", style_function=lambda x: {
    'fillColor': 'green', 'color': 'green', 'weight': 1, 'fillOpacity': 0.3
}).add_to(m)

# Add underserved UHI zones
folium.GeoJson(uhi_without_infra, name="UHI Outside Buffer", style_function=lambda x: {
    'fillColor': 'orange', 'color': 'orange', 'weight': 1, 'fillOpacity': 0.5
}).add_to(m)

folium.LayerControl().add_to(m)
m